# Git III

In [ ]:
import pandas as pd
import ipywidgets as W, traitlets as T, IPython
from wxyz.datagrid import DataGrid
from wxyz.json_schema_form import JSONSchemaForm
from wxyz.dvcs import Git, HeadPicker, Committer, TimeTraveler, HeadStatus

A consistent schema helps keep things work well.

In [ ]:
todo_schema = dict(
    title="Todo List",
    type="object",
    properties=dict(
        tasks=dict(
            default=[],
            type="array",
            items=dict(
                type="object",
                required=["task"],
                properties=dict(
                    task=dict(
                        title="Task",
                        type="string"
                    ),
                    done=dict(
                        title="Done?",
                        type="boolean",
                        default=False
                    )
                )
            )        
        )
    )
)

In [ ]:
def make_a_git_datagrid_example(df=None, schema=None, value=None, record_path=None, work_dir="_git_iii_grid_repo_"):
    schema = schema or todo_schema
    value = value or dict(tasks=[])
    record_path = record_path or "tasks"
    form = JSONSchemaForm(schema=schema, value=value)
    grid = DataGrid(layout=dict(height="300px", flex="1"))
    T.dlink((form, "value"), (grid, "value"), lambda rjsf: pd.json_normalize(rjsf, record_path=record_path))
    T.dlink((grid, "value"), (form, "value"), lambda df: dict(tasks=df.to_dict(orient="records")))
    repo = Git(work_dir)
    repo.track(tracked_widget=form, tracked_traits=["value"], path="form.yaml")
    repo.track(tracked_widget=grid, tracked_traits=["value"], path="df.json")
    status = HeadStatus(repo=repo)
    head = HeadPicker(repo=repo)
    timetravel = TimeTraveler(repo=repo)
    committer = Committer(repo=repo)
    box = W.VBox() 
    box.children = [
        W.HBox([
            form,
            grid,
            W.VBox([head, committer]),
        ]),
        status,
        timetravel, 
    ]
    box._dom_classes += ("jp-wxyz-dvcs-playground-box",)
    return box, repo

In [ ]:
if __name__ == "__main__":
    box, repo = make_a_git_datagrid_example()
    IPython.display.display(box)

In [ ]:
if __name__ == "__main__":
    with __import__("importnb").Notebook():
        from wxyz.notebooks import Utils
        Utils.maybe_log_widget_counts()